# Langchain and RAG Testing

Initialize the model

In [17]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

Asking something in langchain

In [18]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand what love is.")
]

In [19]:
res = chat(messages)
print(res.content)

Love is a complex and powerful emotion that can take many forms. It involves deep feelings of affection, care, and connection towards another person. Love can manifest in different relationships such as romantic love, familial love, and platonic love. It often involves an emotional bond, a sense of intimacy, and a willingness to support and care for the other person. Love can bring joy, fulfillment, and a sense of belonging, but it can also be challenging and require effort and understanding from both parties involved.


To maintain context, you need to append extra messages to the previous chain

In [20]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Do you believe in love?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

As an AI, I don't have personal beliefs or emotions, so I don't have the capacity to believe in love in the way humans do. However, I can provide information and insights on the concept of love based on the knowledge I have been programmed with. Love is a fundamental aspect of human experience, and many people find meaning, fulfillment, and happiness through their relationships with others.


GPT 3.5 is not supposed to have hallucinations, I'm going to ask it something it knows nothing about to test this

In [21]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What do you know about my classmate Ismael Picazo?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [22]:
print(res.content)

I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. My purpose is to provide general information and assistance to the best of my abilities. If you have any specific questions or topics you'd like to discuss, feel free to let me know.


Lets add some more context and tell it who Ismael is

In [23]:
llmchain_information = [
    "Ismael Picazo is a student at the IE University, great guy.",
    "Ismael is known for his work in the field of computer science.",
    "Ismael has a great future ahead of him."
]

source_knowledge = "\n".join(llmchain_information)

Now we feed it the information just like this

In [24]:
query = "Can you tell me about Ismael Picazo?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

In [25]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [26]:
print(res.content)

Based on the provided contexts, Ismael Picazo is a student at IE University known for his work in the field of computer science. He is described as a great guy with a promising future ahead of him.


Importing data

In [27]:
import json
with open('EV_data.json', 'r') as file:
    product_data = json.load(file)


Here is the chatbot answering questions about the data

In [28]:
from langchain.prompts import ChatPromptTemplate

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

embedding_function = OpenAIEmbeddings()

loader = JSONLoader(file_path="./EV_data.json", jq_schema=".electric_vehicles[]", text_content=False)
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
retriever = db.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

query = "What is the seating capacity of the Thunderbolt S?"
print(chain.invoke(query))

The seating capacity of the Thunderbolt S is 4.
